* Python code replication of: " https://www.kaggle.com/janniskueck/pm1-notebook-inference "
* Created by: Anzony Quispe & Alexander Quispe

This notebook contains an example for teaching.

# An inferential problem: The Gender Wage Gap

In the previous lab, we already analyzed data from the March Supplement of the U.S. Current Population Survey (2015) and answered the question how to use job-relevant characteristics, such as education and experience, to best predict wages. Now, we focus on the following inference question:

What is the difference in predicted wages between men and women with the same job-relevant characteristics?

Thus, we analyze if there is a difference in the payment of men and women (*gender wage gap*). The gender wage gap may partly reflect *discrimination* against women in the labor market or may partly reflect a *selection effect*, namely that women are relatively more likely to take on occupations that pay somewhat less (for example, school teaching).

To investigate the gender wage gap, we consider the following log-linear regression model

\begin{align}
\log(Y) &= \beta'X + \epsilon\\
&= \beta_1 D  + \beta_2' W + \epsilon,
\end{align}

where $D$ is the indicator of being female ($1$ if female and $0$ otherwise) and the
$W$'s are controls explaining variation in wages. Considering transformed wages by the logarithm, we are analyzing the relative difference in the payment of men and women.

## Data Analysis

We consider the same subsample of the U.S. Current Population Survey (2015) as in the previous lab. Let us load the data set.



In [ ]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("Plots")
Pkg.add("GLMNet")
Pkg.add("CategoricalArrays")
using GLMNet
using CSV
using DataFrames
using Dates
using Plots
using Statistics
using CategoricalArrays

In [8]:
cd("C:\\Users\\Roberto Carlos\\Documents\\GitHub\\ECO224\\data")

data = CSV.File("wage2015_subsample_inference.csv"; types = Dict("occ" => String,"occ2"=> String,"ind"=>String,"ind2"=>String)) |> DataFrame
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

Number of Rows : 5150
Number of Columns : 21


***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience

In [9]:
describe(data)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,rownames,15636.3,10,15260.0,32643,0,Int64
2,wage,23.4104,3.02198,19.2308,528.846,0,Float64
3,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
4,sex,0.444466,0.0,0.0,1.0,0,Float64
5,shs,0.023301,0.0,0.0,1.0,0,Float64
6,hsg,0.243883,0.0,0.0,1.0,0,Float64
7,scl,0.278058,0.0,0.0,1.0,0,Float64
8,clg,0.31767,0.0,0.0,1.0,0,Float64
9,ad,0.137087,0.0,0.0,1.0,0,Float64


In [10]:
#we have to verify that all the columns has the appropiate data type because we use some categorical variable in this julia_notebooks
[eltype(col) for col = eachcol(data)]

21-element Vector{DataType}:
 Int64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 String
 String
 String
 String

In [11]:
# we can notice that some categorical features now are string so we have to change this 
categorical!(data, [:occ,:occ2,:ind,:ind2])
[eltype(col) for col = eachcol(data)]

LoadError: UndefVarError: categorical! not defined

To start our (causal) analysis, we compare the sample means given gender:

In [12]:
Z = select(data, ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"])

data_female = filter(row -> row.sex == 1, data)
Z_female = select(data_female,["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] )

data_male = filter(row -> row.sex == 0, data)
Z_male = select(data_male,["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] )

means = DataFrame( variables = names(Z), All = describe(Z, :mean)[!,2], Men = describe(Z_male,:mean)[!,2], Female = describe(Z_female,:mean)[!,2])


,variables,All,Men,Female
,String,Float64,Float64,Float64
1,lwage,2.97079,2.98783,2.94948
2,sex,0.444466,0.0,1.0
3,shs,0.023301,0.0318071,0.0126693
4,hsg,0.243883,0.294303,0.180865
5,scl,0.278058,0.273331,0.283967
6,clg,0.31767,0.293953,0.347313
7,ad,0.137087,0.106606,0.175186
8,ne,0.227767,0.22195,0.235037
9,mw,0.259612,0.259,0.260376


In particular, the table above shows that the difference in average logwage between men and women is equal to $0,038$

In [13]:
mean(Z_female[:,:lwage]) - mean(Z_male[:,:lwage])

-0.03834473367441493

Thus, the unconditional gender wage gap is about $3,8$\% for the group of never married workers (women get paid less on average in our sample). We also observe that never married working women are relatively more educated than working men and have lower working experience.

This unconditional (predictive) effect of gender equals the coefficient $\beta$ in the univariate ols regression of $Y$ on $D$:

\begin{align}
\log(Y) &=\beta D + \epsilon.
\end{align}

We verify this by running an ols regression in Julia.

In [ ]:
#install all the package that we can need
Pkg.add("Plots")
Pkg.add("Lathe")
Pkg.add("GLM")
Pkg.add("StatsPlots")
Pkg.add("MLBase")
Pkg.add("Tables")

# Load the installed packages
using DataFrames
using CSV
using Tables
using Plots
using Lathe
using GLM



   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
   Installed StatsFuns ────────────── v0.9.7
   Installed PDMats ───────────────── v0.11.7
   Installed GR_jll ───────────────── v0.64.2+0
   Installed StaticArrays ─────────── v1.4.3
   Installed DSP ──────────────────── v0.6.10
   Installed DoubleFloats ─────────── v1.1.28
   Installed ProgressMeter ────────── v1.7.2
   Installed PlotUtils ────────────── v1.2.0
   Installed Roots ────────────────── v1.4.0
   Installed RecipesPipeline ──────── v0.5.2
   Installed MLBase ───────────────── v0.8.0
   Installed StructuralCausalModels ─ v1.0.8
   Installed GPUArrays ────────────── v8.3.1
   Installed Polynomials ──────────── v1.2.1
   Installed GLM ──────────────────── v1.6.2
   Installed DataFrames ───────────── v0.22.7
   Installed EllipsisNotation ─────── v1.5.0
 

In [ ]:
nocontrol_model = lm(@formula(lwage ~ sex),data)
nocontrol_est = GLM.coef(nocontrol_model)[2]
nocontrol_se = GLM.coeftable(nocontrol_model).cols[2][2]

println("The estimated gender coefficient is ", nocontrol_est ," and the corresponding robust standard error is " ,nocontrol_se )

Next, we run an ols regression of $Y$ on $(D,W)$ to control for the effect of covariates summarized in $W$:

\begin{align}
\log(Y) &=\beta_1 D  + \beta_2' W + \epsilon.
\end{align}

Here, we are considering the flexible model from the previous lab. Hence, $W$ controls for experience, education, region, and occupation and industry indicators plus transformations and two-way interactions.

Let us run the ols regression with controls.

## Ols regression with controls

In [ ]:
flex = @formula(lwage ~ sex + (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))
control_model = lm(flex , data)
control_est = GLM.coef(control_model)[2]
control_se = GLM.coeftable(control_model).cols[2][2]
println(control_model)
println("Coefficient for OLS with controls " , control_est)



The estimated regression coefficient $\beta_1\approx-0.0696$ measures how our linear prediction of wage changes if we set the gender variable $D$ from 0 to 1, holding the controls $W$ fixed.
We can call this the *predictive effect* (PE), as it measures the impact of a variable on the prediction we make. Overall, we see that the unconditional wage gap of size $4$\% for women increases to about $7$\% after controlling for worker characteristics.  


Next, we are using the Frisch-Waugh-Lovell theorem from the lecture partialling-out the linear effect of the controls via ols.

## Partialling-Out using ols

In [ ]:
# models
# model for Y
flex_y = @formula(lwage ~ (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))
flex_d = @formula(sex ~ (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))

# partialling-out the linear effect of W from Y
t_Y = residuals(lm(flex_y, data))

# partialling-out the linear effect of W from D
t_D = residuals(lm(flex_d, data))

data_res = DataFrame(t_Y = t_Y, t_D = t_D )
# regression of Y on D after partialling-out the effect of W
partial_fit = lm(@formula(t_Y ~ t_D), data_res)
partial_est = GLM.coef(partial_fit)[2]

println("Coefficient for D via partiallig-out ", partial_est)

# standard error
partial_se = GLM.coeftable(partial_fit).cols[2][2]

#condifence interval
GLM.confint(partial_fit)[2,:]

Again, the estimated coefficient measures the linear predictive effect (PE) of $D$ on $Y$ after taking out the linear effect of $W$ on both of these variables. This coefficient equals the estimated coefficient from the ols regression with controls.

We know that the partialling-out approach works well when the dimension of $W$ is low
in relation to the sample size $n$. When the dimension of $W$ is relatively high, we need to use variable selection
or penalization for regularization purposes. 

In the following, we illustrate the partialling-out approach using lasso instead of ols. 

## Summarize the results

In [ ]:
DataFrame(modelos = [ "Without controls", "full reg", "partial reg" ], 
Estimate = [nocontrol_est,control_est,partial_est], 
StdError = [nocontrol_se,control_se, partial_se])

It it worth to notice that controlling for worker characteristics increases the gender wage gap from less that 4\% to 7\%. The controls we used in our analysis include 5 educational attainment indicators (less than high school graduates, high school graduates, some college, college graduate, and advanced degree), 4 region indicators (midwest, south, west, and northeast);  a quartic term (first, second, third, and fourth power) in experience and 22 occupation and 23 industry indicators.

Keep in mind that the predictive effect (PE) does not only measures discrimination (causal effect of being female), it also may reflect
selection effects of unobserved differences in covariates between men and women in our sample.


Next we try "extra" flexible model, where we take interactions of all controls, giving us about 1000 controls.

## "Extra" flexible model

In [ ]:
import Pkg
Pkg.add("StatsModels")
Pkg.add("Combinatorics")
Pkg.add("IterTools")
# we have to configure the package internaly with the itertools package, this because 
#julia dont iunderstand (a formula) ^2, it takes as an entire term not as interactions 
#between variables

In [ ]:
#this code fix the problem mencioned above
using StatsModels, Combinatorics, IterTools

combinations_upto(x, n) = Iterators.flatten(combinations(x, i) for i in 1:n)
expand_exp(args, deg::ConstantTerm) =
    tuple(((&)(terms...) for terms in combinations_upto(args, deg.n))...)

StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.Schema, ctx::Type) =
    apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.FullRank, ctx::Type) =
    apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

In [ ]:
extra_flex = @formula(lwage ~  sex + (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2)

control_fit = lm(extra_flex, data)
control_est = GLM.coef(control_fit)[2]

println("Number of Extra-Flex Controls: ", size(modelmatrix(control_fit))[2] -1) #minus the intercept
println("Coefficient for OLS with extra flex controls ", control_est)

#std error
control_se = GLM.stderror(control_fit)[2];